# 🌐✨ **AI-Powered Language Translator with Gemini 1.5**

Welcome to the **AI-Powered Language Translator** project!  
This solution leverages the power of **🔮 Google Gemini 1.5 Flash** for real-time language translation.  
We integrate **LangChain** for smart orchestration and **Gradio** to deliver a clean, interactive web interface.

💬 Features:
- 🌍 Automatic **language detection**
- 🔁 Real-time **translation**
- 📝 Persistent **translation history**

---

## 📚 Table of Contents

Navigate through the key components of this notebook:

1. 🔧 [Installation](#installation)  
2. 🔐 [API Key Setup](#api-key-setup)  
3. ⚙️ [Environment Setup](#environment-setup)  
4. 🌍 [Language Detection & Translation](#language-detection-translation)  
5. 🖼️ [Building the UI with Gradio](#building-the-ui-with-gradio)  
6. 🚀 [Running the Application](#running-the-application)  
7. ✅ [Conclusion](#conclusion)

---

## 📦 1. Installation - Prepare the Environment

🔧 Let's start by installing all the essential packages required for this project. This includes tools for:

- ✅ Working with Google's Generative AI (`google-generativeai`)
- 🔗 Integrating with LangChain (`langchain`, `langchain-google-genai`, `langchain_community`)
- 🖼️ Building interfaces with `gradio`
- 🌍 Detecting language using `langdetect`

> 💡 Note: Make sure you are running this in a Python 3 environment with internet access.

In [ ]:
!pip install google-generativeai langchain langchain-google-genai langchain_community gradio langdetect

##🔑 2. API Key Setup - Set the Access Key for Google Gemini

In this step, we'll securely configure the **API key** required for accessing **Google Gemini 1.5 Flash**.  
We will use **Colab's userdata** for safely storing and retrieving the key.

### 🛠️ 2.1 Initialize API Key for Google Gemini

To get started, we'll load the **Gemini API key** into the environment variable.  
This ensures that the key is securely available for use without hardcoding it directly into the code.

In [ ]:
# Import necessary library for Colab's userdata storage
from google.colab import userdata

# Securely retrieve the Gemini API Key from the userdata
GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')  # Ensure the key is stored in Colab's userdata

# Set the Gemini API key as an environment variable for secure access
import os
os.environ["GOOGLE_API_KEY"] = GEMINI_API_KEY  # Replace with your own API Key

##⚙️ 3. Environment Setup - Prepare the Libraries

In this section, we'll import all the necessary libraries that will power our translation system. These libraries handle everything from translation tasks to creating a user interface.


###📚 3.1 Import Libraries

We will import the following essential libraries:

- **Gradio** (`gr`) for building the user interface
- **LangChain** and **GoogleGenerativeAI** to interface with Google's Gemini API for translations
- **LangChain Prompt Templates** for generating dynamic prompts
- **LangChain Core** for running language models and processing outputs
- **Langdetect** for detecting the language of input text
- **JSON** for handling structured data exchange

In [ ]:
# Import Gradio for building the interface
import gradio as gr

# Import GoogleGenerativeAI from LangChain for translation functionality
from langchain_google_genai import GoogleGenerativeAI

# Import LangChain's PromptTemplate for creating structured prompts
from langchain.prompts import PromptTemplate

# Import core components from LangChain
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# Import langdetect for language detection
from langdetect import detect

# Import JSON for handling structured data
import json

###🧠 3.2 Initialize the Language Model - Set up Google Gemini

In this step, we will initialize the **GoogleGenerativeAI model** from LangChain.  
This model will be used to handle the language translation tasks, utilizing **Gemini 1.5 Flash** for fast and accurate translations.

We configure the model with the following settings:
- **Model**: `"gemini-1.5-flash-latest"` (latest version of Gemini 1.5)
- **Temperature**: `0.7` (controls the randomness of the generated responses; 0.7 provides a balanced output)


In [ ]:
# Initialize the GoogleGenerativeAI model from LangChain for translation tasks
llm = GoogleGenerativeAI(model="gemini-1.5-flash-latest", temperature=0.7)

## 🌍 4. Language Detection & Translation - Detect Language and Translate

In this section, we'll focus on creating a dynamic translation system.  
We'll define a **dynamic prompt** that adapts based on the language input and the desired translation language.

###📝 4.1 Create a Dynamic Translation Prompt - Build a Flexible Template

Here, we will create a **multi-language prompt template** to handle dynamic translation tasks.  
The prompt will be structured to take any input text and translate it into the specified language.

The template format is:
- **Input variables**: `["text", "language"]`  
- **Template**: "Translate the following sentence to {language}:\n\n{text}\n\nThe translation is:"


In [ ]:
# Create a dynamic prompt template for translation tasks
prompt = PromptTemplate(
    input_variables=["text", "language"],
    template="""
You are a professional translator.

Translate the following sentence into {language}, and then briefly explain the meaning of the sentence in a human-friendly way.

Sentence:
{text}

Output format:
- Translated Sentence: <translated_sentence>
- Explanation: <short explanation in target language>
"""
)

###🔗 4.2 Build LangChain Pipeline - Define the Translation Process

In this step, we'll define the **LangChain pipeline** that will orchestrate the translation process.  
The pipeline will handle the input text, target language, and translation request, passing through each component seamlessly.

The pipeline includes:
1. **Input text** and **language** are passed through **RunnablePassthrough** components.
2. The **prompt template** is applied to create a dynamic translation query.
3. The **Google Gemini model** (LLM) processes the prompt and generates the translation.
4. The **output** is parsed into a readable format using **StrOutputParser**.

In [ ]:
# Build the LangChain pipeline for translation tasks
chain = (
    {"text": RunnablePassthrough(), "language": RunnablePassthrough()}  # Input handlers for text and language
    | prompt  # Apply the translation prompt
    | llm  # Generate translation with the language model
    | StrOutputParser()  # Parse and format the output translation
)

### 🌐 4.3 Language Detection Function - Automatically Detect the Source Language

In this step, we will implement a function to **automatically detect the language** of the input text using the `langdetect` library.  
This function will help identify the source language before translating it to the target language.

The function will:
1. Take the input **text** as a parameter.
2. Use the `langdetect.detect()` method to identify the language.
3. Handle any errors and return the exception message if detection fails.

In [ ]:
# Function to detect the language of the input text
def detect_language(text):
    try:
        # Automatically detect the language of the input text
        return detect(text)
    except Exception as e:
        # Return error message if language detection fails
        return str(e)

### 📝 4.4 Save Translation History - Store Translation Data

In this step, we will create a function to **save the translation history**.  
The history will be stored in a **JSON file** to keep a record of the translations, including the original text, target language, and translation output.

The function will:
1. Create a dictionary to hold the **text**, **language**, and **translation**.
2. Append this data to a JSON file (`translation_history.json`) for future reference.
3. Each entry will be saved in a new line for easy parsing.

In [ ]:
# Function to save the translation history to a JSON file
def save_translation_history(text, language, translation):
    # Create a dictionary with the translation details
    history = {"text": text, "language": language, "translation": translation}

    # Open the history file in append mode and store the translation data
    with open('translation_history.json', 'a') as file:
        json.dump(history, file)  # Save the data as a JSON object
        file.write("\n")  # Ensure each entry is on a new line

##🎨 5. Building the UI with Gradio - Create the Translation Interface

In this section, we will define the **main translation function** that integrates the input text, language detection, and translation process.  
The function will:
1. Accept **text** and **target language** as input.
2. Automatically detect the **source language** of the text.
3. Pass the text and target language through the **LangChain pipeline** to generate the translation.
4. Save the translation to the **history** for future reference.
5. Return the **source language** and **translated text** as output.

###🧑‍💻 5.1 Define the Translation Function - Main Logic

In [ ]:
# Main translation function that integrates text input, language detection, and translation chain
def translate_text(text, language):
    # Check if both text and target language are provided
    if not text or not language:
        return "Both text and target language must be provided."

    try:
        # Automatically detect the source language of the input text
        source_language = detect_language(text)

        # Prepare input data for LangChain pipeline
        input_data = {"text": text, "language": language}

        # Invoke the translation chain to get the translated text
        result = chain.invoke(input_data)

        # Save the translation history to a JSON file
        save_translation_history(text, language, result)

        # Return source language and translated result
        return f"Source Language: {source_language}\n\nTranslated Text:\n{result}"

    except Exception as e:
        # Return error message if any exception occurs
        return f"An error occurred: {str(e)}"

###🎨 5.2 Define Gradio Interface - Create the UI for Translation Tool

In this step, we will create the **Gradio UI** for the translation tool.  
The user interface will include:
1. **Text input** for users to enter sentences in any language.
2. A **dropdown menu** for selecting the **target language**.
3. The translation result will be displayed in a **textbox** output.

The interface is designed to be intuitive and easy to use, allowing users to input text, select the language, and view the translation in real-time.


In [ ]:
# List of supported languages for dropdown menu
languages = [
    "English", "Indonesian", "Japanese", "Arabic",
    "French", "Spanish", "German", "Other (type manually)"
]

# Define the Gradio interface using Blocks for better layout customization
with gr.Blocks() as demo:
    # Main title and short description
    gr.Markdown("# 🌍 Multilingual Sentence Translator")
    gr.Markdown(
        """
        This tool uses the **Gemini 1.5 Flash** model to translate sentences into different languages.
        It supports **automatic language detection** and also allows users to manually specify a target language
        that is not listed in the dropdown menu.
        """
    )

    # Input section: text to translate
    text_input = gr.Textbox(
        label="Enter sentence (any language)",
        lines=2,
        placeholder="Type a sentence to translate..."
    )

    # Language selection row: dropdown + optional custom input
    with gr.Row():
        dropdown_lang = gr.Dropdown(
            choices=languages,
            label="Select Target Language",
            value="English"
        )
        custom_lang = gr.Textbox(
            label="Or enter custom language",
            placeholder="e.g., Korean, Italian, Thai",
            interactive=False  # Disable by default
        )

    # Button to translate and area to show the result
    translate_btn = gr.Button("Translate")
    output_box = gr.Textbox(
        label="Translation Result",
        lines=6
    )

    # Function to control whether custom_lang is enabled
    def toggle_custom_input(choice):
        if choice == "Other (type manually)":
            return gr.Textbox(interactive=True)
        else:
            return gr.Textbox(value="", interactive=False)

    # Translation logic
    def handle_translate(text, dropdown, custom):
        target_language = custom if dropdown == "Other (type manually)" else dropdown
        return translate_text(text, target_language)

    # Link dropdown changes to enabling/disabling the textbox
    dropdown_lang.change(
        fn=toggle_custom_input,
        inputs=dropdown_lang,
        outputs=custom_lang
    )

    # Link translate button to translation logic
    translate_btn.click(
        fn=handle_translate,
        inputs=[text_input, dropdown_lang, custom_lang],
        outputs=output_box
    )

##🚀 6. Running the Application - Launch the Translator UI

Now that we have built the translation tool and its user interface, it's time to run the application!  
To launch the app and interact with the **language translator**, simply execute the cell below. Gradio will start the UI, allowing users to input text, select a target language, and view the translated result in real-time.


In [ ]:
# Launch the Gradio interface
demo.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://2a3ca416deccd48f27.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


## 🏁 7. Conclusion - Recap of the Translation Tool

This project showcases the integration of **Google Gemini 1.5 Flash**, **LangChain**, and **Gradio** to build a powerful and interactive **multi-language translator**.  
The system provides several key features:
1. **Automatic source language detection** to identify the language of the input text.
2. **Real-time translation** to convert the text into the target language.
3. **Translation history saving** to keep track of past translations for future reference.

The **Gradio UI** provides an intuitive and user-friendly interface, making the entire translation process smooth and efficient.

💡 **Key Takeaways:**  
- **Google Gemini 1.5 Flash** delivers accurate and fast translations.
- **LangChain** handles the orchestration of the translation pipeline.
- **Gradio** makes it easy to create interactive UIs for machine learning applications.

This system demonstrates how to build a complete, end-to-end translation tool, combining cutting-edge technologies to create a seamless experience for users.